# Θ-Extraction – Batch Run for Cuprates
Materials: LSCO x=0.15, YBCO, Bi2212, CaCuO₂ (infinite-layer)

Pipeline per material: σ(ω,T) → M(ω) → π(ω) → Θ(T) → Θ_c/T_c
Gates: KK, Method Agreement, Classification, ε-sensitivity


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from extract_theta_from_optical import (
    MATERIALS, generate_dummy_data, compute_memory_function,
    spectral_measure, extract_theta, extract_theta_full_pipeline
)

print("✓ Imports ready")

## 1) Helper: temperature grid per material

In [ ]:
def temp_grid(Tc):
    arr = np.array([0.5, 0.7, 0.9, 1.1, 1.5, 2.0])*Tc
    return np.unique(arr.astype(int))

## 2) Batch evaluation

In [ ]:
materials = ['LSCO_x015','YBCO','Bi2212','CaCuO2']
records = []
gate_matrix_rows = []

for key in materials:
    mat = MATERIALS[key]
    Ts = temp_grid(mat.Tc)
    # Full pipeline (canonical) across T-array
    res = extract_theta_full_pipeline(key, Ts, use_dummy=True, energy_method='canonical', verbose=False)

    # KK gate at ~1.1 Tc (single-T diagnostic)
    T_demo = int(max(Ts.min(), min(int(1.1*mat.Tc), Ts.max())))
    w, s1, s2 = generate_dummy_data(key, T_demo, omega_max=5.0, n_points=600)
    M1, M2, Mdiag = compute_memory_function(w, s1, s2, mat.omega_p)
    kk_vio = Mdiag.get('kk_violation', np.nan)
    kk_ok = bool(Mdiag.get('kk_ok', True))

    # ε-sensitivity at T_demo
    thetas = []
    for em in ['canonical','memory','corrected']:
        th, _ = extract_theta(w, spectral_measure(w, s1, method='f-sum'), M1, energy_method=em)
        thetas.append(th)
    thetas = np.array(thetas, dtype=float)
    sens = float(np.std(thetas)/np.mean(thetas)) if np.isfinite(thetas).all() and np.mean(thetas)!=0 else np.nan
    sens_ok = bool(sens < 0.10) if np.isfinite(sens) else False

    # Method agreement on Θ_c
    agree = float(res['Theta_c_diagnostics']['agreement'])
    agree_ok = bool(agree < 0.10)

    # Classification/validation
    validation = res['validation']
    class_ok = bool(validation['within_tolerance'])
    overall = bool(kk_ok and sens_ok and agree_ok and class_ok)

    rec = {
        'Material': mat.name,
        'Structure': mat.structure,
        'Tc_K': float(res['Tc']),
        'Theta_c_eV': float(res['Theta_c']),
        'Theta_c_over_Tc': float(res['ratio']),
        'Expected_ratio': float(MATERIALS[key].expected_ratio),
        'RelError': float(validation['relative_error']),
        'KK_violation': float(kk_vio) if kk_vio==kk_vio else np.nan,
        'KK_pass': bool(kk_ok),
        'Method_agreement': float(agree),
        'Method_pass': bool(agree_ok),
        'Sensitivity': float(sens) if sens==sens else np.nan,
        'Sensitivity_pass': bool(sens_ok),
        'Class_pass': bool(class_ok),
        'Validation_status': validation['status'],
        'Overall_PASS': bool(overall)
    }
    records.append(rec)

    gate_matrix_rows.append({
        'Material': mat.name,
        'Gate: KK': bool(kk_ok),
        'Gate: Method': bool(agree_ok),
        'Gate: ε-sensitivity': bool(sens_ok),
        'Gate: Class': bool(class_ok),
        'OVERALL': bool(overall)
    })

batch_df = pd.DataFrame(records)
matrix_df = pd.DataFrame(gate_matrix_rows)
batch_df

## 3) PASS/FAIL Matrix

In [ ]:
matrix_df

## 4) Save CSVs + summary report

In [ ]:
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
batch_csv = f"theta_batch_summary_{ts}.csv"
matrix_csv = f"theta_batch_gates_{ts}.csv"
batch_df.to_csv(batch_csv, index=False)
matrix_df.to_csv(matrix_csv, index=False)
print("Saved:", batch_csv, "and", matrix_csv)

# Generate a simple aggregated text report
lines = []
lines.append("Θ-Extraction Batch Report")
lines.append(f"Date: {datetime.now()}")
lines.append("")
for r in records:
    lines.append(f"- {r['Material']} | Struct={r['Structure']} | Tc={r['Tc_K']:.1f} K | Θc/Tc={r['Theta_c_over_Tc']:.3f} | "
                 f"KK={'PASS' if r['KK_pass'] else 'FAIL'} | Method={'PASS' if r['Method_pass'] else 'FAIL'} | "
                 f"ε-sens={'PASS' if r['Sensitivity_pass'] else 'FAIL'} | Class={'PASS' if r['Class_pass'] else 'FAIL'} | "
                 f"OVERALL={'PASS' if r['Overall_PASS'] else 'FAIL'}")
report_txt = f"theta_batch_report_{ts}.txt"
with open(report_txt, "w") as f:
    f.write("\n".join(lines))
print("Saved:", report_txt)

batch_csv, matrix_csv, report_txt

## 5) One-figure PASS/FAIL overview (counts)

In [ ]:
import matplotlib.pyplot as plt

counts = {
    'KK': matrix_df['Gate: KK'].sum(),
    'Method': matrix_df['Gate: Method'].sum(),
    'ε-sensitivity': matrix_df['Gate: ε-sensitivity'].sum(),
    'Class': matrix_df['Gate: Class'].sum(),
    'OVERALL': matrix_df['OVERALL'].sum()
}
labels = list(counts.keys())
values = [counts[k] for k in labels]

plt.figure(figsize=(6,4))
plt.bar(labels, values)
plt.ylabel('PASS count')
plt.title('Batch PASS counts (4 materials)')
plt.xticks(rotation=30)
plt.tight_layout()
plt.savefig("theta_batch_pass_counts.png", dpi=150, bbox_inches='tight')
plt.show()